In [ ]:
# Load images into image vector matrix - where each row is an image vector
import os

import glob
import cv2
import numpy as np

# Path templates
category_directories_template =  "food_resized/*/[0-9]*_*"
image_files_template = "food_resized/*/*/*.jpg"

#  return cid, name for a given category directory path i.e. 
# 'food_resized/train/0_hot_dog' ---> 0, "hot_dog"
#       use: cid, name = cid_name_from_path(path_string)
def cname_cid_from_path(path_string):
    
    base_dir = None
    if os.path.isfile(path_string):
        base_dir = os.path.basename(os.path.split(path_string)[0])
    elif os.path.isdir(path_string):
        base_dir = os.path.basename(path_string)
    else: 
        raise("Error incorect path")
    
    cid = int(base_dir.split('_')[0]) # get leading number, or cid
    
    idx = base_dir.find('_') + 1      # get trailing string after first '_'
    cname = base_dir[idx:]

    return cname, cid

# {'chicken_wings': 0, 'dumplings': 1, 'hot_dog': 2, 'ramen': 3, 'sushi': 4}
def get_category_cname_cid_map(category_directories_template):
    category_folders = glob.glob(category_directories_template)
    return dict(map(cname_cid_from_path, category_folders))

def image_to_vector(image):
    return image.reshape((1, -1)) / 255

# image_vector of original size 60, 60, 3 back to np image
def image_vector_to_image(image_vector, original_image_size=(60,60,3)):
    image_vector = image_vector * 255
    image_vector = image_vector.astype(int)
    image_vector = image_vector.reshape(original_image_size)
    return image_vector

def get_sample_images_and_labels(images_file_template, gray=False):
    
    image_paths = glob.glob(image_files_template)
    
    images = []
    labels = []
    
    for fname in image_paths:
        
        label, cid = cname_cid_from_path(fname)
        
        img = None
        if gray == True:
            img = cv2.imread(fname,0)
        else:
            img = cv2.imread(fname)
        
            # color stored in incorrect order for plt -> convert to RGB order
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        images.append(img)
        labels.append(label)
        
    return images, labels

#load all images into matrix of image vectors, X, and corrresponding vector of categories, y
def get_sample_image_vector_matrix_and_targets(images, labels, cname_cid_dict):
    
    X = image_to_vector(images[0])
    y = []

    for img, l in zip(images, labels):
        
        v = image_to_vector(img) # convert image to vector
        X = np.vstack([X,v])     # add image vector to matrix
        
        y.append(cname_cid_dict[l])
        
    y = np.array(y)
    
    return X,y

#
# load all images into matrix of image vectors, X, and corrresponding vector of categories, y
#

cname_cid_dict = get_category_cname_cid_map(category_directories_template) # get cname:cid dictonary
images, labels = get_sample_images_and_labels(image_files_template, gray=False)

% time X,y = get_sample_image_vector_matrix_and_targets(images, labels, cname_cid_dict)  

# save original shape of images
original_image_shape = images[0].shape

In [ ]:
# above operation takes a long time save results to files
import pickle
pickle.dump(X, open( 'large_data/X.p', 'wb' ))
pickle.dump(y, open( 'large_data/y.p', 'wb' ))

In [ ]:
# load
import pickle
X = pickle.load(open( 'large_data/X.p', 'rb' ))
y = pickle.load(open( 'large_data/y.p', 'rb' ))

In [ ]:
idx = 0
v = X[idx]

image = image_vector_to_image(v, original_image_size=original_image_shape) #reconstruct image from vector
plt.imshow(image)

In [ ]:
help(cv2.xfeatures2d.SIFT_create())
help(cv2.xfeatures2d)

In [ ]:
import math
import time

def modified_sigmoid(x, k):
     return 1 / (1 + math.exp(-x/k)) - 0.5

def to_gray(image):
    # converting image to grayscale
    # must save the image again to use cv2.convert color, path must be relative
    local_path = 'tmp.jpg'
    cv2.imwrite(local_path, image)
    img = cv2.imread(local_path)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray


def sift_distance(gray1, gray2, original=None):

    img1 = gray1
    img2 = gray2

    # ORB Detector
    #orb = cv2.ORB_create()
    #kp1, des1 = orb.detectAndCompute(img1, None)
    #kp2, des2 = orb.detectAndCompute(img2, None)

    # Brute Force Matching
    #bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    #matches = bf.match(des1, des2)
    #matches = sorted(matches, key = lambda x:x.distance)
    
    # find the keypoints and descriptors with SIFT
    Sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = Sift.detectAndCompute(img1,None)
    kp2, des2 = Sift.detectAndCompute(img2,None)
    
    # Brute Force Matching
    bf = cv2.BFMatcher()
    
    matches = bf.knnMatch(des1,des2, k=2)

    #Apply ratio test
    threshold = 0.88
    good_matches = 0
    for m,n in matches:
        if m.distance < threshold * n.distance:
            good_matches += 1

    return modified_sigmoid(good_matches, 5)


# number of images
n = X.shape[0]

# distance matrix
D = np.zeros(shape=(n,n))


start = time.time()
for i in range(n):
    print("%s \% complete" % (i/n))
    for j in range(n):
        
        idx1= i
        idx2 = j

        im1 = image_vector_to_image(X[idx1])
        im2 = image_vector_to_image(X[idx2])
        gray1 = to_gray(im1)
        gray2 = to_gray(im2)

        sd = sift_distance(gray1, gray2)
        
        D[i,j] = sd
        D[j,i] = sd

        
# set diagnols to zero
np.fill_diagonal(D, 0)
end = time.time()
print("time: %s" % (end - start))

print(D)

# above operation takes a long time save results to files
import pickle
pickle.dump(X, open( 'large_data/D.p', 'wb' ))

In [ ]:
# load
import pickle
D = pickle.load(open( 'large_data/D.p', 'rb' ))